In [1]:
import os
import sys
import torch
sys.path.append("/opt/ml/level2_objectdetection-cv-09/")
from utils.im_utils import Compose, ToTensor, RandomHorizontalFlip
from config.train_config import cfg, Config
from dataloader.dataset import CustomDataset, split_train_valid, get_all_annotation, get_json_data
from utils.train_utils import create_model
from torchsummary import summary

In [2]:
device = torch.device(cfg.device_name)
print("Using {} device training.".format(device.type))

if not os.path.exists(cfg.model_save_dir):
    os.makedirs(cfg.model_save_dir)

data_transform = {
    "train": Compose([ToTensor(), RandomHorizontalFlip(cfg.train_horizon_flip_prob)]),
    "val": Compose([ToTensor()])
}

if not os.path.exists(cfg.data_root_dir):
    raise FileNotFoundError("dataset root dir not exist!")
json_data = get_json_data(cfg.data_root_dir)

json_anno = get_all_annotation(json_data)

train_info, val_info = split_train_valid(cfg.data_root_dir, json_data)

# load train data set
train_data_set = CustomDataset(train_info, json_anno, data_transform['train'])
batch_size = cfg.batch_size
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])
print('Using {} dataloader workers'.format(nw))
train_data_loader = torch.utils.data.DataLoader(train_data_set,
                                                batch_size=batch_size,
                                                shuffle=True,
                                                num_workers=nw,
                                                collate_fn=train_data_set.collate_fn)

# load validation data set
val_data_set = CustomDataset(val_info, json_anno, data_transform['val'])
val_data_set_loader = torch.utils.data.DataLoader(val_data_set,
                                                    batch_size=batch_size,
                                                    shuffle=False,
                                                    num_workers=nw,
                                                    collate_fn=train_data_set.collate_fn)

# create model num_classes equal background + 80 classes
model = create_model(num_classes=cfg.num_class)

Using cuda device training.
Using 8 dataloader workers
box_head None
<class 'torchvision.ops.poolers.MultiScaleRoIAlign'>


In [3]:
img,target = train_data_set.__getitem__(2)

In [4]:
print(model)
# summary(model.train(), input_size=(3,1024,1024),device='cpu')

FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, af

In [5]:
model.train().cuda()
for images, targets in train_data_loader:
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    output = model(images, targets)
    break

RuntimeError: shape '[1376256, -1]' is invalid for input of size 5586944